# GPT 4o-mini

In [ ]:
import os
from openai import OpenAI
from PIL import Image
import base64
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm  
import logging

# set log
log_dir = './results/gpt4o_mini/'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, 'prediction_log.txt')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[
    logging.FileHandler(log_file),
    logging.StreamHandler()
])
logger = logging.getLogger()

# Load data
df_table_prd = pd.read_csv('LMM_sewerML/results/df_table_prd.csv')
df_table_dsc = pd.read_csv('LMM_sewerML/results/df_table_dsc.csv')
image_dir = 'LMM_sewerML/images'

# set openai key, need to fill in
client = OpenAI(api_key="")

# Define the prompt template
prompt_template = """
You are a virtual sewer inspection technician equipped to analyze images from CCTV cameras taken inside sewer pipes. Your primary task is to examine each image and classify specific defect types present in the sewer pipeline, as per the provided defect codes. You should carefully evaluate the image and assess which, if any, defects appear.

For each image, you will:
1. Provide a brief, precise description of the scene inside the pipe.
2. Identify any defect types observed from the following codes, specifying the code and its severity level based on your assessment.

The list of defect types and their descriptions:
- CIW: VA - Water level, measured in percentage.
- RB: Cracks, breaks, and collapses.
- OB: Surface damage.
- PF: Production error.
- DE: Deformation.
- FS: Displaced joint.
- IS: Intruding sealing material.
- RO: Roots.
- IN: Infiltration.
- AF: Settled deposits.
- BE: Attached deposits.
- FO: Obstacle.
- GR: Branch pipe.
- PH: Chiseled connection.
- PB: Drilled connection.
- OS: Lateral reinstatement cuts.
- OP: Connection with transition profile.
- OK: Connection with construction changes.

Provide the output in JSON format as follows:
- DESCRIPTION: "<Description of the pipe interior and any observed features>"
- DEFECTS: [
     { "CODE": "<Defect Code>", "DESCRIPTION": "<Defect Description>", "SEVERITY": "<Assessed Severity Level>" }
]

If no defect is detected, set "CODE" to "NoDefect", with an empty "DESCRIPTION" and "SEVERITY" fields.
"""

# Store predictions and actual labels
predictions = []
actual_labels = []
descriptions = []

# Iterate over each row in the ground truth dataframe with progress bar
for idx, row in tqdm(df_table_prd.iterrows(), total=df_table_prd.shape[0], desc="Processing images"):
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        logger.warning(f"Image {img_id} not found. Skipping.")
        continue

    # Load image
    with open(img_path, "rb") as image_file:
        img_b64_str = base64.b64encode(image_file.read()).decode('utf-8')
    img_type = "image/png"  # set image type

    # Create prompt with image placeholder
    prompt = prompt_template

    # OpenAI GPT-4o
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:{img_type};base64,{img_b64_str}"}},
                ],
            }
        ],
    )

    # Extract defect prediction and description from model output
    # print(response)
    response_text = response.choices[0].message.content.strip()
    
    if '"CODE": "NoDefect"' in response_text:
        predicted_defect = "NoDefect"
        desc_start_idx = response_text.find('"DESCRIPTION": "') + len('"DESCRIPTION": "')
        desc_end_idx = response_text.find('"', desc_start_idx)
        defect_description = response_text[desc_start_idx:desc_end_idx]
    else:
        # Extract defect code from JSON output in response
        start_idx = response_text.find('"CODE": "') + len('"CODE": "')
        end_idx = response_text.find('"', start_idx)
        predicted_defect = response_text[start_idx:end_idx]

        # Extract defect description from JSON output in response
        desc_start_idx = response_text.find('"DESCRIPTION": "') + len('"DESCRIPTION": "')
        desc_end_idx = response_text.find('"', desc_start_idx)
        defect_description = response_text[desc_start_idx:desc_end_idx]

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)
    descriptions.append(defect_description)

    # Log intermediate outputs
    logger.info(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}")

# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

logger.info(f"\nAccuracy: {accuracy:.4f}")
logger.info(f"F1 Score: {f1:.4f}")
logger.info(f"\nClassification Report:\n{report}")

# Save results to CSV files
df_table_prd['gpt4o-mini'] = predictions
df_table_prd.to_csv('./results/gpt4o_mini/df_table_prd.csv', index=False)

df_table_dsc['gpt4o'] = descriptions
df_table_dsc.to_csv('./results/gpt4o_mini/df_table_dsc.csv', index=False)

Processing images:   0%|          | 0/200 [00:00<?, ?it/s]2024-11-12 00:51:45,849 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 00:51:45,860 - INFO - Image ID: 00617545.png | Predicted: RB | Ground Truth: RB
Processing images:   0%|          | 1/200 [00:05<16:43,  5.04s/it]2024-11-12 00:51:50,004 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 00:51:50,007 - INFO - Image ID: 00635967.png | Predicted: BE | Ground Truth: NoDefect
Processing images:   1%|          | 2/200 [00:09<14:54,  4.52s/it]2024-11-12 00:51:55,466 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-12 00:51:55,469 - INFO - Image ID: 00002720.png | Predicted: CIW | Ground Truth: NoDefect
Processing images:   2%|▏         | 3/200 [00:14<16:14,  4.95s/it]2024-11-12 00:51:58,251 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-1